In [1]:
from ultralytics import YOLO
from PIL import Image, ImageDraw 
import cv2 
import numpy as np
import torch
from torchvision.transforms import InterpolationMode, transforms
import timm
from IPython.display import display

# Define classes
txt_animalclasses = {
    'en': ["badger", "ibex", "red deer", "chamois", "cat", "goat", "roe deer", "dog", "squirrel", "equid", "genet",
           "hedgehog", "lagomorph", "wolf", "lynx", "marmot", "micromammal", "mouflon",
           "sheep", "mustelid", "bird", "bear", "nutria", "fox", "wild boar", "cow"],
}


In [2]:
class Detector:
    def __init__(self):
        self.model = YOLO('../models/deepfaune-yolov8s_960.pt')

    def bestBoxDetection(self, imagecv):
        image_rgb = cv2.cvtColor(imagecv, cv2.COLOR_BGR2RGB)
        image_pil = Image.fromarray(image_rgb)
        resized_image = image_pil.resize((960, 960), Image.Resampling.LANCZOS)
        results = self.model(resized_image)

        if not results or not results[0].boxes or results[0].boxes.data.shape[0] == 0:
            return None, 0, np.zeros(4), 0, None

        detections = results[0].boxes.data
        best_detection = detections[detections[:, 4].argmax()]
        xmin, ymin, xmax, ymax, conf, cls_id = best_detection[:6]
        box = [int(xmin), int(ymin), int(xmax), int(ymax)]
        cropped_image = resized_image.crop(box)
        return cropped_image, int(cls_id), box, conf, None

In [3]:
class Classifier:
    def __init__(self, device):
        self.model = self.load_model('../models/deepfaune-vit_large_patch14_dinov2.lvd142m.pt', device)
        self.transforms = transforms.Compose([
            transforms.Resize((182, 182), interpolation=InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def load_model(self, model_path, device):
        model = timm.create_model('vit_large_patch14_dinov2', pretrained=False, num_classes=len(txt_animalclasses['en']), dynamic_img_size=True)
        state_dict = torch.load(model_path, map_location=torch.device(device))['state_dict']
        adjusted_state_dict = {k.replace('base_model.', ''): v for k, v in state_dict.items() if 'base_model.' in k}
        model.load_state_dict(adjusted_state_dict)
        return model

    def predict(self, image):
        img_tensor = self.transforms(image).unsqueeze(0)
        with torch.no_grad():
            output = self.model(img_tensor)
            probabilities = torch.nn.functional.softmax(output, dim=1)
            top_p, top_class = probabilities.topk(1, dim=1)
            return txt_animalclasses['en'][top_class.item()], top_p.item()


In [4]:
def visualize_detection(cropped_image, label, confidence):
    draw = ImageDraw.Draw(cropped_image)
    text = f"{label} ({confidence*100:.0f}%)"
    draw.text((10, 10), text, fill="red")
    display(cropped_image)

def process_single_image(image_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    image = Image.open(image_path)
    imagecv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    detector = Detector()
    classifier = Classifier(device)
    cropped_image, category, box, conf, _ = detector.bestBoxDetection(imagecv)

    if cropped_image is not None:
        animal_type, confidence = classifier.predict(cropped_image)
        print(f"{device}: Detection with confidence {conf:.3f}, Classification of {animal_type} with confidence {confidence:.3f}")
        visualize_detection(cropped_image, animal_type, confidence)
    else:
        print("No object detected.")

current_image_path = "/home/tom-ratsakatika/Downloads/maybe a fox.jpg"

process_single_image(current_image_path)


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 6.81 MiB is free. Process 6614 has 2.63 GiB memory in use. Including non-PyTorch memory, this process has 1.17 GiB memory in use. Of the allocated memory 1.10 GiB is allocated by PyTorch, and 640.00 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
!nvidia-smi

Wed May  8 11:59:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   53C    P0              19W /  35W |   1531MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--